### ETL - FATO GRAO

In [ ]:
### Funcao Carregar Dados fato_grao
def carregar_dados_fato_grao(driver, server, database,dados, tabela_destino):
    
    con = conexao(driver, server, database)
  
    query = f'''INSERT INTO {tabela_destino} ''' + '''(''' + ','.join(list(dados)) + ''')''' + ''' values(''' + str('?,' * dados.shape[1])[:-1] + ''')'''
    
    cursor = con.cursor()
    print("Query Feita")
    
    try:
        for index, row in dados.iterrows():
            cursor.execute(query, row.DataLaudo, row.IdPedido, row.IdExame, row.Material, row.IdUnidade, row.IdConvenio, row.IdUnidadeProdutiva, row.IdUnidadeNegocio, row.SITUACAO, row.ValorProducao)
        con.commit()
        cursor.close()
    except:
        print('Erro na Inserção dos Dados')
        
    return

In [ ]:
### FATO GRAO

query = """SELECT DataLaudo, -- PK
        IdPedido, -- PK
        IdExame, -- PK
        Material, -- PK
        IdUnidade, -- PK
        IdConvenio,
        IdUnidadeProdutiva,
        IdUnidadeNegocio,
        SITUACAO,
        ValorProducao FROM DATASET_1 """

try:
    new_data = pd.read_sql(query, con)
except:
    print("Erro na Leitura dos Dados")

if len(new_data) == 0:
    print("Tabela Vazia")
else:
    new_data['SITUACAO'] = new_data['SITUACAO'].astype(str) 
    new_data.drop_duplicates(keep = 'first', inplace = True)
    try:
        truncate_table(driver, server, database, 'fato_grao')
        print("Tabela Truncada")
    except:
        print("Erro no Truncamento da Tabela")
    carregar_dados_fato_grao(driver, server, database, new_data, 'fato_grao')
    print("Insersao Concluida")